**당뇨병환자 분류**

In [1]:
import numpy as np
import tensorflow as tf
tf.set_random_seed(777)

In [6]:
xy = np.loadtxt('DataSet/diabetes.csv', delimiter = ',', dtype = np.float32)
print(xy)

[[-0.294118   0.487437   0.180328  ... -0.53117   -0.0333333  0.       ]
 [-0.882353  -0.145729   0.0819672 ... -0.766866  -0.666667   1.       ]
 [-0.0588235  0.839196   0.0491803 ... -0.492741  -0.633333   0.       ]
 ...
 [-0.411765   0.21608    0.180328  ... -0.857387  -0.7        1.       ]
 [-0.882353   0.266332  -0.0163934 ... -0.768574  -0.133333   0.       ]
 [-0.882353  -0.0653266  0.147541  ... -0.797609  -0.933333   1.       ]]


In [7]:
xdata = xy[:,0:-1]
ydata = xy[:,[-1]]
print(xdata.shape, ydata.shape)

(759, 8) (759, 1)


In [15]:
x = tf.placeholder(tf.float32, shape = [None, 8])
y = tf.placeholder(tf.float32, shape = [None, 1])
w = tf.Variable(tf.random_normal([8,1]))
b = tf.Variable(tf.random_normal([1]))
hf = tf.sigmoid(tf.matmul(x,w) + b)
cost = -tf.reduce_mean(y*tf.log(hf) + (1-y)*tf.log(1-hf))
train = tf.train.GradientDescentOptimizer(0.01).minimize(cost)
predicted = tf.cast(hf>0.5, tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted,y), dtype = tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for step in range(10001):
        cv, _ = sess.run([cost, train], feed_dict = {x:xdata, y:ydata})
        if step % 500 == 0 :
            print(step, cv)
            
    cv, av = sess.run([predicted, accuracy], feed_dict = {x:xdata, y:ydata})
    print("정확도 : ", av)

0 0.78797144
500 0.69929034
1000 0.6541983
1500 0.61999065
2000 0.59363246
2500 0.57318795
3000 0.55719835
3500 0.5445714
4000 0.5344939
4500 0.5263617
5000 0.5197259
5500 0.51425225
6000 0.5096898
6500 0.5058494
7000 0.5025868
7500 0.4997911
8000 0.49737608
8500 0.49527448
9000 0.49343288
9500 0.49180865
10000 0.4903676
정확도 :  0.76943344


* * *

In [20]:
sess = tf.Session()
myqueue = tf.train.string_input_producer(['DataSet/q_1.txt','DataSet/q_2.txt','DataSet/q_3.txt'], shuffle=False) #입력데이터를 자동으로 만들어주는 함수
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess = sess, coord = coord) # coord, threads 수식 항상 고정

reader = tf.TextLineReader()
key, value = reader.read(myqueue)


print(sess.run(key))
print(sess.run(value))
rd = [[0],[0]]
for i in range(100) :
    speed, dist = tf.decode_csv(value, record_defaults=rd)
    if i % 10 == 0 :
        print(sess.run([speed, dist]))
coord.request_stop()
coord.join(threads=threads)

b'DataSet/q_1.txt:1'
b','
[4, 10]
[0, 0]
[7, 4]
[7, 22]
[8, 16]
[9, 10]
[10, 18]
[10, 26]
[10, 34]
[11, 17]


In [25]:
sess = tf.Session()
myqueue = tf.train.string_input_producer(['DataSet/q_1.txt','DataSet/q_2.txt','DataSet/q_3.txt'], shuffle=False) #입력데이터를 자동으로 만들어주는 함수



reader = tf.TextLineReader()
key, value = reader.read(myqueue)
record_defaults = [[-1],[999]]
speed, dist = tf.decode_csv(value, record_defaults=record_defaults)
x_batch, y_batch = tf.train.batch([speed, dist], batch_size = 4)

coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess = sess, coord = coord) # coord, threads 수식 항상 고정

for i in range(100) :
    x, y = sess.run([x_batch, y_batch])
    # x와 y를 이용하여 모델링
#     print(x,y)
coord.request_stop()
coord.join(threads=threads)

